In [1]:
# make display wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

In [44]:
# read in packages
%matplotlib inline
import pandas as pd
import json
import geopandas as gpd
import numpy as np
from shapely import wkt
import os
from keplergl import KeplerGl

In [45]:
# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

In [46]:
#
usa_df = pd.read_csv(r'data/dest_mnxUSco_daily_01-09-2020.csv')
# alter file, convert geoid field to string
usa_df = usa_df.rename(columns={'geoid':'id'})
usa_df['id'] = usa_df['id'].str[1:]
list_date = usa_df.columns.to_list()
list_date = list_date[1:]  

dff = pd.melt(usa_df, id_vars=['id'],value_vars = list_date)

dff['variable'] = pd.to_datetime(dff['variable'])
dff.head()

,id,variable,value
0,01001,2020-01-01,0.000000
1,01003,2020-01-01,7.785615
2,01005,2020-01-01,1.557123
3,01007,2020-01-01,0.000000
4,01009,2020-01-01,0.000000


In [47]:
# read in counties GeoJSON
counties = r'data/shapefiles/UScounties.geojson'
counties = gpd.read_file(counties)
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"MULTIPOLYGON (((-89.18137 37.04630, -89.17938 ..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"MULTIPOLYGON (((-84.44266 38.28324, -84.44114 ..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"MULTIPOLYGON (((-86.94486 37.07341, -86.94346 ..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"MULTIPOLYGON (((-84.12662 37.64540, -84.12483 ..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"MULTIPOLYGON (((-83.98428 38.44549, -83.98246 ..."


In [48]:
counties['geometry'] = counties.centroid
counties = counties.set_geometry('geometry')
#counties.head()
counties.to_file("data/shapefiles/UScounties_centroid.geojson", driver="GeoJSON")

C:\Users\steve\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [49]:
# add independent lat/longs for KeplerGl
counties['lon'] = counties['geometry'].x
counties['lat'] = counties['geometry'].y
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,lon,lat
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,POINT (-84.21715 38.20673),-84.217151,38.206735
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,POINT (-86.68162 37.20728),-86.681623,37.207285
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,POINT (-83.96431 37.69244),-83.964311,37.692444
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,POINT (-83.69666 38.37012),-83.696656,38.370118


In [50]:
usa_centroid = pd.merge(counties, dff, left_on="GEOID", right_on="id", how="inner")
usa_centroid.to_csv(r'data/usa_cntr_jansep.csv')

In [51]:
usa_centroid.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,lon,lat,id,variable,value
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482,21007,2020-01-01,0.0
1,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482,21007,2020-01-02,0.0
2,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482,21007,2020-01-03,0.0
3,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482,21007,2020-01-04,0.0
4,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,POINT (-88.99926 37.05848),-88.999256,37.058482,21007,2020-01-05,0.0


In [52]:
array = ['New York', 'Kings','Queens','Richmond','Bronx']
nyc = usa_centroid.loc[(usa_centroid['STATEFP'] == '36') & usa_centroid['NAME'].isin(array)]
#usa_centroid = usa_centroid.loc[(usa_centroid['STATEFP'] != '36') & (df['NAME'] != 'New York', 'Kings','Queens','Richmond','Bronx')]
nyc.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,lon,lat,id,variable,value
46035,36,047,00974122,0500000US36047,36047,Kings,06,179671325,71098973,POINT (-73.93853 40.63954),-73.938527,40.639541,36047,2020-01-01,7415.020009
46036,36,047,00974122,0500000US36047,36047,Kings,06,179671325,71098973,POINT (-73.93853 40.63954),-73.938527,40.639541,36047,2020-01-02,6106.591241
46037,36,047,00974122,0500000US36047,36047,Kings,06,179671325,71098973,POINT (-73.93853 40.63954),-73.938527,40.639541,36047,2020-01-03,6240.631164
46038,36,047,00974122,0500000US36047,36047,Kings,06,179671325,71098973,POINT (-73.93853 40.63954),-73.938527,40.639541,36047,2020-01-04,6144.853151
46039,36,047,00974122,0500000US36047,36047,Kings,06,179671325,71098973,POINT (-73.93853 40.63954),-73.938527,40.639541,36047,2020-01-05,5469.264803


In [53]:
len(usa_centroid)

898659

In [54]:
usa_centroid.tail()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,lon,lat,id,variable,value
898654,26,139,01623012,0500000US26139,26139,Ottawa,06,1459502408,2765830983,POINT (-85.99610 42.95985),-85.996101,42.959849,26139,2020-10-01,0.000000
898655,26,139,01623012,0500000US26139,26139,Ottawa,06,1459502408,2765830983,POINT (-85.99610 42.95985),-85.996101,42.959849,26139,2020-10-02,0.000000
898656,26,139,01623012,0500000US26139,26139,Ottawa,06,1459502408,2765830983,POINT (-85.99610 42.95985),-85.996101,42.959849,26139,2020-10-03,1.737287
898657,26,139,01623012,0500000US26139,26139,Ottawa,06,1459502408,2765830983,POINT (-85.99610 42.95985),-85.996101,42.959849,26139,2020-10-04,0.000000
898658,26,139,01623012,0500000US26139,26139,Ottawa,06,1459502408,2765830983,POINT (-85.99610 42.95985),-85.996101,42.959849,26139,2020-10-05,0.000000


In [55]:
# remoing the NYC counties 
nyc_geoid = ['36005','36047','36061','36081','36085']
usa_centroid = usa_centroid[~usa_centroid['GEOID'].isin(nyc_geoid)]
len(usa_centroid)

897264

In [56]:
# Aleutian Islands gives odd geography -- let's just remove from analysis
# Get names of indexes for which column Age has value 30
indexNames = usa_centroid[usa_centroid['NAME'] == 'Aleutians West' ].index
# Delete these row indexes from dataFrame
usa_centroid.drop(indexNames, inplace=True)
len(usa_centroid)

896985

In [69]:
df_centroid = pd.read_csv(r'data/values_cntr_jansep.csv')
usa = 'usa'
subplaces = 'subplaces'
decimals = 0
def kepler_func(df,geo_name):
    df['date'] = pd.to_datetime(df['variable'])
    df['date'] = df["date"].dt.strftime('%m/%d/%Y')
    df['datetime'] = df['date'].astype(str) + ' 0:00'
    #df_centroid.drop(columns = 'geometry')
    #df['geom'] = df['geom'].apply(wkt.loads)
    df['value'] = df['value'].astype(float)
    df['value'] = df['value'].apply(lambda x: round(x, decimals)) # round to nearest whole num
    df['value'] = df['value'].astype(int) # remove decimal
    # selecting rows based on value being greater than 0
    df = df.loc[df['value'] >= 1] 
    #renaming tooltip columns for Kepler.gl hover
    df = df.rename(columns = {'NAME':'Name','date':'Date','value':'Trips/Device'})
    #df_centroid = gpd.GeoDataFrame(df_centroid, geometry='geom')
    df.to_csv(f'data/kepler_{geo_name}.csv', index = False)

C:\Users\steve\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
# read geographies into the function
kepler_func(df_centroid, subplaces)
kepler_func(usa_centroid, usa)

In [13]:
config = {
'version': 'v1',
'config': {
    'mapState': {
        'latitude': 40.7137,
        'longitude': -73.9675,
        'zoom': 12.3
    }}}

In [14]:
map1 = KeplerGl(height = 300)
map1.add_data(data=df_centroid,name ='df')
map1.config = config

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


ValueError: Can't clean for JSON: <shapely.geometry.point.Point object at 0x000002908F9D4208>